<a href="https://colab.research.google.com/github/sxy031145/PracticeForML/blob/master/algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

N = 21;
num_cluster = 6;
num_iteration = 100;
size_cluster = np.zeros(num_cluster);
C_l = np.zeros((num_cluster, N));
flag_clus = np.zeros((num_cluster, N));
#I_k = 
num_node = np.zeros(N);
N_k = np.zeros((N, N));
flag_node = np.zeros((N, N));
rd = np.random.RandomState(114514);
A_para = rd.random((num_cluster, num_cluster));

In [2]:
graph = np.loadtxt("graph.txt");
num_edge = (int)(graph.size/2);
for i in range(num_edge):
  u = (int)(graph[i*2-1]);
  v = (int)(graph[i*2]);
  N_k[u][(int)(num_node[u])] = v;
  N_k[v][(int)(num_node[v])] = u;
  num_node[u] = num_node[u] + 1;
  num_node[v] = num_node[v] + 1;
  flag_node[u][v] = 1;
  flag_node[v][u] = 1;
cluster = np.loadtxt("cluster.txt");
pos = 0;
for i in range(num_cluster-1):
  size_cluster[i] = cluster[pos];
  pos = pos + 1;
  for j in range((int)(size_cluster[i])):
    C_l[i][j] = cluster[pos];
    flag_clus[i][(int)(cluster[pos])] = 1;
    pos = pos + 1;

C_l


array([[ 2.,  5., 10., 11., 16., 17., 19.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 3.,  5.,  6., 11., 14., 15., 20.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 2.,  4.,  5.,  8., 12., 16.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  2., 10., 13., 16., 19.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 7.,  9., 10., 16., 17., 18., 19.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [5]:

def cal_A_para():
  for i in range(num_cluster-1):
    for j in range(1, N):
      for k in range(1, N):
        if flag_node[j][k] > 0 and flag_clus[i][k] > 0:
          num_l[i][j] = num_l[i][j] + 1;

  for i in range(num_cluster-1):
    for j in range(N-1):
      for k in range(N-1):
        if j == k:
          A_l[i][j][k] = 1.0;
          continue;
        if flag_node[j][k] > 0 and flag_clus[i][k] > 0:
          if max(num_l[i][j+1], num_l[i][k+1]) == 0:
            A_l[i][j][k] = 1.0;
          else:
            A_l[i][j][k] = 1.0/max(num_l[i][j+1], num_l[i][k+1]);
    for j in range(1, N):
      for k in range(1, N):
        if j == k:
          continue;
        A_l[i][j][j] = A_l[i][j][j] - A_l[i][j][k];
      if A_l[i][j][j] < 0:
        A_l[i][j][j] = 0;


In [6]:
A_l = np.zeros((num_cluster, N, N));
num_l = np.zeros((num_cluster, N));
cal_A_para();

In [7]:
def cal_Omega(k):
  #A_tmp = np.zeros((N-1, N-1));
  #for i in range(1, N):
  #  for j in range(1, N):
  #    A_tmp[i-1][j-1] = A_l[l][i][j];
  #eig, feature = np.linalg.eig(A_tmp);
  #nearEig = 0;
  #minLength = 1.0*10000;
  #for i in range(eig.size):
  #  if minLength > abs(eig[i] - 1.0):
  #    minLength = abs(eig[i] - 1.0);
  #    nearEig = i;
  Omega = np.zeros((num_cluster-1, num_cluster-1));
  for i in range(1, N-1):
    if flag_node[i][k] == 0:
      continue;
    for j in range(num_cluster-1):
      if flag_clus[j][i] > 0:
        Omega[j][j] = Omega[j][j] + 1.0;
  return Omega;

print(cal_Omega(2));

[[1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 0.]]


In [8]:
import math
def fun_g_EP(x):
  return x*x;

def fun_h_IP(x, rho):
  return abs(x*x*x/math.sqrt(x*x + rho*rho));

def fun_g_EP_grad(x):
  return 2*x;

def fun_h_IP_grad(x, rho):
  tmp = x*x*(2*x*x+3*rho*rho)/math.sqrt(x*x+rho*rho);
  return tmp / (x*x+rho*rho);

def cal_p(x, rho):
  ans = np.zeros(num_cluster-1);
  for i in range(num_cluster-1):
    ans[i] = fun_g_EP(x[i]) + fun_h_IP(x[i], rho);
  return ans;

def cal_p_grad(x, rho):
  ans = np.zeros(num_cluster-1);
  for i in range(num_cluster-1):
    ans[i] = fun_g_EP_grad(x[i]) + fun_h_IP_grad(x[i], rho);
  return ans;

In [ ]:

print(w);

In [9]:
def Coupled_Diffusion(w, mu, eta, rho, num_iteration):
  Omega = np.zeros((N, num_cluster-1, num_cluster-1));
  xi = np.zeros(num_cluster-1);
  phi = np.zeros((N-1, num_cluster-1));
  for i in range(1,N-1):
    Omega[i] =cal_Omega(i);
  #print(Omega);
  for i in range(num_iteration):
    for k in range(1,N-1):
      #print(w[k]);
      tmp = cal_p_grad(w[k], rho);
      print(tmp);
      for j in range(num_cluster-1):
        tmp[j] = tmp[j]*Omega[k][j][j]*mu*eta;
      xi = w[k] - tmp;
      tmp = 2*xi;
      for j in range(num_cluster-1):
        tmp[j] = tmp[j]*Omega[k][j][j]*mu;
      phi[k] = xi - tmp;
    print(phi);
    for j in range(num_cluster-1):
      for k in range(1,N-1):
        w[k][j] = 0.0;
        for s in range(1,N-1):
          if A_l[j][s][k] <= 0:
            continue;
          w[k][j] = w[k][j] + A_l[j][s][k]*phi[k][j];

    print(w);
  return w;
    

In [10]:
w = rd.random((N, num_cluster-1));
Coupled_Diffusion(w, 0.01, 0.1, 0.5, 1);

[2.70122377 0.56889872 3.4359859  0.08015471 2.17532958]
[3.81424956 1.59808896 2.24616521 2.17755187 0.38725094]
[1.92637325 0.4367224  0.30200273 2.33044195 2.45895694]
[0.53851211 0.08632981 0.10647693 2.8725271  3.32271029]
[1.44567082 0.03083502 1.87424481 2.45896481 2.74700983]
[2.39299409 0.1748446  2.96570373 2.4345408  1.04516667]
[1.767866   0.29399628 0.1915683  2.24469759 3.81274135]
[1.72460227 3.83634547 1.98295807 1.47968765 3.83185872]
[0.39456233 1.18229752 3.34668048 3.17117152 2.43526312]
[2.47784091 0.13134853 2.52473667 1.69608609 1.34766381]
[1.31731943 0.28901461 0.71416109 0.52096521 2.3996296 ]
[3.76183798 0.20433631 1.95130108 0.07159411 2.39954668]
[2.25562001 0.4257521  1.93295997 0.38637785 3.78981322]
[0.1136407  0.28108859 2.29397284 2.46204997 3.47841596]
[1.81335994 1.43567208 2.99568224 0.6246598  0.69947456]
[2.12273033 3.39756374 1.39040295 0.41852424 3.2461634 ]
[0.04160323 2.68825174 2.89595509 0.59909592 0.28512308]
[3.24304321 0.45938649 2.852130

In [ ]:
pp = rd.random(5);
print(cal_p_grad(pp, 0.5));